# Предсказание категории объявлений на Авито.

Для начала импортируем все необходимые модули и загружаем данные.

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv('train.csv')
data.head()

,item_id,title,description,price,category_id
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


Здесь определяем функцию, которая будет очищать и преобразовывать текст. Она объединяет текст с заголовком, приводит его к нижнему регистру и очищает от не буквенных символов.

In [3]:
def transform_text(text_data):
    reg = re.compile('[^а-яА-Яa-zA-Z ]')
    X_new=[]
    for index,row in text_data.iterrows():
        X_new.append(row['title']+" "+row['description'])
    for i in range(len(X_new)):    
        X_new[i]=X_new[i].lower()
        X_new[i]=reg.sub('',X_new[i])
    return(X_new)        

Дальше выделяем выборку и целевую переменную, и трансформируем выборку.

In [4]:
X=data.drop(['item_id','category_id'],axis=1)
y=data['category_id'].values

In [5]:
X_train_new=transform_text(X)

Затем векторизуем текст по методу TF-IDF. Так же учитываем сочетания 2-х и 3-х слов для векторизации.

In [6]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_features=40000,ngram_range=(1,3))
data_vectorized = vectorizer.fit_transform(X_train_new)

Здесь нормализуем цену для дальнейшего соединения с матрицей.

In [7]:
X_ = StandardScaler()
price=X['price'].values
price=price.reshape(-1, 1)
price_std = X_.fit_transform(price)

Теперь объединяем матрицу с ценой для формирования единой матрицы признаков.

In [8]:
X_data=hstack((data_vectorized,price_std))

Проверяем выбранную модель на кросс-валидации.

In [9]:
kf = model_selection.KFold(n_splits=5,shuffle=True,random_state=22)
logreg=LogisticRegression(solver='lbfgs',C=5, multi_class='auto')
sc=model_selection.cross_val_score(logreg, X_data, y,cv=kf,scoring='accuracy')
print(f"Score:{sc.mean()}")

Score:0.8369923972645417


Преобразуем целевую переменную так, чтобы учитывались только 4 наиболее общие категории и проверяем качество модели. Логично, что качество оказывается выше, так как классов становится меньше.

In [10]:
y_second=[]
for i in y:
    if i in range(0,15):
        y_second.append(0)
    elif i in range(15,30):
        y_second.append(1)
    elif i in range(30,42):
        y_second.append(2)
    elif i in range(42,54):
        y_second.append(3)

In [11]:
sc=model_selection.cross_val_score(logreg, X_data, y_second,cv=kf,scoring='accuracy')
print(f"Score:{sc.mean()}")

Score:0.9429580564613168


Теперь аналогично преобразуем для менее общих категорий, после которых разбиение будет возможно только на самые мелкие классы.

In [12]:
y_third=[]
for i in y:
    if i in [0,2,6,9,10,11]:
        y_third.append(0)
    elif i == 1:
        y_third.append(1)
    elif i == 3:
        y_third.append(2)
    elif i in [4,8]:
        y_third.append(3)
    elif i in [5,12]:
        y_third.append(4)
    elif i == 7:
        y_third.append(5)
    elif i in [13,14]:
        y_third.append(6)
    elif i in [15,17,21,25]:
        y_third.append(7)
    elif i in [16,18,19,22,23,27,28]:
        y_third.append(8)
    elif i == 20:
        y_third.append(9)
    elif i == 24:
        y_third.append(10)
    elif i in [26,29]:
        y_third.append(11)
    elif i in [30,32,34,41]:
        y_third.append(12)
    elif i in [31,33,35,38,39]:
        y_third.append(13)
    elif i in [36,40]:
        y_third.append(14)
    elif i == 37:
        y_third.append(15)
    elif i in [42,46,47]:
        y_third.append(16)
    elif i in [43,51]:
        y_third.append(17)
    elif i in [44,45]:
        y_third.append(18)
    elif i == 48:
        y_third.append(19)
    elif i == 49:
        y_third.append(20)
    elif i in [50,52]:
        y_third.append(21)
    elif i == 53:
        y_third.append(22)

Проверяем качество для таких классов. Оно оказывается между предыдущими двумя, как и ожидалось.

In [13]:
sc=model_selection.cross_val_score(logreg, X_data, y_third,cv=kf,scoring='accuracy')
print(f"Score:{sc.mean()}")

Score:0.9049062720890342


Обучаем модель.

In [14]:
model=logreg.fit(X_data,y)

Загружаем тестовые данные.

In [15]:
test_data=pd.read_csv('test.csv')
X_test=test_data.drop(['item_id'],axis=1)

Трасформируем, векторизуем и объединяем тестовые данные.

In [16]:
X_test_new=transform_text(X_test)

In [17]:
test_data_vectorized = vectorizer.transform(X_test_new)

In [19]:
test_price=X_test['price'].values
test_price=test_price.reshape(-1, 1)
test_price_std = X_.transform(test_price)

In [20]:
X_test_data=hstack((test_data_vectorized,test_price_std))

Делаем предсказание и записываем ответы в отдельный файл.

In [21]:
y_pred=model.predict(X_test_data)

In [24]:
item_id=test_data['item_id'].values
df_item=pd.DataFrame(item_id,index=None,columns=['item_id'])
df_category=pd.DataFrame(y_pred,index=None,columns=['category_id'])
df=df_item.join(df_category)
df.to_csv('test_answer.csv',index=False)